<a href="https://colab.research.google.com/github/aderdouri/CQFDev/blob/master/Module_01/binomial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step-by-Step Guide to Pricing Options Using the Binomial Model

### Introduction
The binomial model is a discrete-time method for pricing options. It simulates the possible movements of the underlying asset price over multiple time steps and calculates the option price through backward induction.

### Step 1: Define the Model Parameters}
The required parameters for the binomial tree are:
- Current stock price: $S_0$
- Strike price: $K$
- Risk-free interest rate: $r$
- Volatility: $\sigma$
- Time to maturity: $T$
- Number of time steps: $N$

The time step size is computed as:
\begin{equation}
    \Delta t = \frac{T}{N}
\end{equation}

### Step 2: Construct the Binomial Price Tree
The up ($u$) and down ($d$) factors are defined as:
\begin{equation}
    u = e^{\sigma \sqrt{\Delta t}}
\end{equation}
\begin{equation}
    d = \frac{1}{u} = e^{-\sigma \sqrt{\Delta t}}
\end{equation}
The risk-neutral probability is given by:
\begin{equation}
    p = \frac{e^{r \Delta t} - d}{u - d}
\end{equation}
The stock price at each node $(i, j)$ is computed as:
\begin{equation}
    S_{i,j} = S_0 \cdot u^j \cdot d^{i-j}
\end{equation}

### Step 3: Compute Option Payoff at Maturity
At expiration ($t = T$), the option value is determined by its intrinsic value:
- European call option:
\begin{equation}
    V_N^j = \max(S_{N,j} - K, 0)
\end{equation}
- European put option:
\begin{equation}
    V_N^j = \max(K - S_{N,j}, 0)
\end{equation}

### Step 4: Backward Induction for Option Pricing
We work backward from the terminal nodes to the initial node. The option price at each node is computed as:
\begin{equation}
    V_i^j = e^{-r \Delta t} \cdot \left( p \cdot V_{i+1}^{j+1} + (1 - p) \cdot V_{i+1}^{j} \right)
\end{equation}
For American options, the value at each node is the maximum of the intrinsic value and the continuation value:
\begin{equation}
    V_i^j = \max(\text{Intrinsic Value}, \text{Continuation Value})
\end{equation}

### Step 5: Extract the Option Price
The option price at time $t = 0$ is the value at the root node:
\begin{equation}
    V_0^0
\end{equation}
This represents the fair value of the option under the binomial model.

## Conclusion
The binomial model provides a simple yet powerful way to price options, allowing for flexibility in handling American options and accommodating different market conditions.

In [9]:
import numpy as np

def binomial_tree(S0, K, T, r, sigma, num_steps, is_call=True, is_american=False):
    """
    Price an option using the binomial tree method.

    Parameters:
    - S0: Initial stock price (float)
    - K: Strike price (float)
    - T: Time to maturity (float, in years)
    - r: Risk-free rate (float)
    - sigma: Volatility (float)
    - num_steps: Number of steps in the binomial tree (int)
    - is_call: Whether the option is a call (default True). False for a put option.
    - is_american: Whether the option is American (default False). True for American option.

    Returns:
    - option_price: float
    """
    delta_t = T / num_steps
    u = np.exp(sigma * np.sqrt(delta_t))  # Up factor
    d = 1 / u  # Down factor
    p = (np.exp(r * delta_t) - d) / (u - d)  # Risk-neutral probability
    q = 1 - p  # Complement probability

    # Initialize asset price tree
    S = np.zeros((num_steps + 1, num_steps + 1))
    S[0, 0] = S0
    for i in range(1, num_steps + 1):
        S[i, 0] = S[i - 1, 0] * u
        for j in range(1, i + 1):
            S[i, j] = S[i - 1, j - 1] * d

    # Initialize option value tree
    V = np.zeros((num_steps + 1, num_steps + 1))
    for j in range(num_steps + 1):
        if is_call:
            V[num_steps, j] = max(S[num_steps, j] - K, 0)  # Call option payoff
        else:
            V[num_steps, j] = max(K - S[num_steps, j], 0)  # Put option payoff

    # Backward induction
    for i in range(num_steps - 1, -1, -1):
        for j in range(i + 1):
            continuation_value = np.exp(-r * delta_t) * (p * V[i + 1, j] + q * V[i + 1, j + 1])
            if is_american:
                if is_call:
                    intrinsic_value = max(S[i, j] - K, 0)
                else:
                    intrinsic_value = max(K - S[i, j], 0)
                V[i, j] = max(intrinsic_value, continuation_value)
            else:
                V[i, j] = continuation_value

    return V[0, 0]

# Example usage
if __name__ == "__main__":
    num_steps = 200
    S0 = 10
    K = 10
    sigma = 0.4
    r = 0.05
    T = 1

    call_price = binomial_tree(S0, K, T, r, sigma, num_steps, is_call=True, is_american=True)
    put_price = binomial_tree(S0, K, T, r, sigma, num_steps, is_call=False, is_american=True)
    print(f"American Call Option Price: {call_price:.6f}")
    print(f"American Put Option Price: {put_price:.6f}")

American Call Option Price: 1.800349
American Put Option Price: 1.365733
